In [1]:
import pickle
import pandas as pd

#Load the row paragraphs
with open('labeledData.pickle', 'rb') as handle:
    data = pickle.load(handle)    
#Load the labels
labels = pd.read_excel('labels.xls', sheet_name = None)
del labels['TEMPLATE SHEET']
del labels['megaresistencia.com']
del labels['Ahaishopping.com']
del labels['massprofits.com']
del labels['rachelrayshow.com']

In [6]:
for filename in labels.keys():
    print('Loading '+filename)
    if filename+'.md' in data.keys():
        print('---- '+filename+'.md'+" found in data source")
        for paraTuple in data[filename+'.md']:
            paraIndex = paraTuple[0]
            paragraph = paraTuple[1]
            paragraphProcesses = " ".join(list(filter(lambda a: a != '\n', paragraph)))        
            labels[filename].loc[paraIndex, 'PARAGRAPH'] = paragraphProcesses
        labels[filename]['PARAGRAPH'] = labels[filename]['PARAGRAPH'].fillna('')
        labels[filename]['LABEL'] = labels[filename]['LABEL'].fillna(0)
        labels[filename]['AMBIGUITY'] = labels[filename]['AMBIGUITY'].fillna(0)

Loading jetpack.me
---- jetpack.me.md found in data source
Loading integralads.com
---- integralads.com.md found in data source
Loading cell.com
---- cell.com.md found in data source
Loading amiestreet.com
---- amiestreet.com.md found in data source
Loading 99u.com
---- 99u.com.md found in data source
Loading dialogflow.com
---- dialogflow.com.md found in data source
Loading dynatrace.com
---- dynatrace.com.md found in data source
Loading facebook.com
---- facebook.com.md found in data source
Loading cedato.com
---- cedato.com.md found in data source
Loading islamtoleran.com
---- islamtoleran.com.md found in data source
Loading insidecrm.com
---- insidecrm.com.md found in data source
Loading autodesk.eu
---- autodesk.eu.md found in data source
Loading githubstatus.com
---- githubstatus.com.md found in data source
Loading springserve.com
---- springserve.com.md found in data source
Loading themcrookedvultures.com
---- themcrookedvultures.com.md found in data source
Loading icanhascheezb

In [12]:
labels['facebook.com'].loc[16, 'PARAGRAPH']

'How can I manage or delete information about me? We provide you with the ability to access, rectify, port and erase your data. Learn more in your Facebook Settings and Instagram Settings.  We store data until it is no longer necessary to provide our services and Facebook Products, or until your account is deleted - whichever comes first. This is a case-by-case determination that depends on things like the nature of the data, why it is collected and processed, and relevant legal or operational retention needs. For example, when you search for something on Facebook, you can access and delete that query from within your search history at any time, but the log of that search is deleted after 6 months. If you submit a copy of your government-issued ID for account verification purposes, we delete that copy 30 days after submission. Learn more about deletion of content you have shared and cookie data obtained through social plugins. '

In [9]:
labels['facebook.com'].loc[16, 'PARAGRAPH']

'How can I manage or delete information about me? We provide you with the ability to access, rectify, port and erase your data. Learn more in your Facebook Settings and Instagram Settings.  We store data until it is no longer necessary to provide our services and Facebook Products, or until your account is deleted - whichever comes first. This is a case-by-case determination that depends on things like the nature of the data, why it is collected and processed, and relevant legal or operational retention needs. For example, when you search for something on Facebook, you can access and delete that query from within your search history at any time, but the log of that search is deleted after 6 months. If you submit a copy of your government-issued ID for account verification purposes, we delete that copy 30 days after submission. Learn more about deletion of content you have shared and cookie data obtained through social plugins. '

In [10]:
#save DB
with open('DB.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
example = "How HugeDomains Collects this Information HugeDomains utilizes a variety of security measures to maintain the safety of customers' personal information. All supplied financial information (including credit card data) is transmitted via Secure Socket Layer (SSL) technology and then encrypted into the company's payment gateway provider's database. This database and the sensitive information contained within are only accessible by those authorized with special access rights to such systems and are required to keep the information confidential. "

In [32]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/patrickzoechbauer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
def calc_bestfit(example, labels):
    bestfit = dict() 
    bestfit['similarity'] = 0
    for file in labels: 
        print(file)
        for para in range(len(labels[file])):
            similarity = cosine_sim(example, labels[file].loc[para,'PARAGRAPH'])        
            if bestfit['similarity'] < similarity:
                #new best fit
                bestfit['similarity'] = similarity
                bestfit['file'] = file
                bestfit['label'] = labels[file].loc[para, 'LABEL']
                bestfit['PARAGRAPH'] = labels[file].loc[para, 'PARAGRAPH']
    return bestfit

In [34]:
calc_bestfit(example, labels)

jetpack.me
integralads.com


/opt/anaconda3/envs/LawCS/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_wor

cell.com
amiestreet.com
99u.com
dialogflow.com
dynatrace.com
facebook.com
cedato.com
islamtoleran.com
insidecrm.com
autodesk.eu
githubstatus.com
springserve.com
themcrookedvultures.com
icanhascheezburger.com
perezitos.com
statdx.com
tradetracker.com
autodesk.in
bitsbeta.com
buzzfudge.com
fidelity.com
postimage.io
quickprivacycheck.com
puterdolls.com
sun.com
telexads.com
pjeshka.com
thefusejoplin.com
signal.co
telechargervideoyoutube.com


{'similarity': 1.0,
 'file': 'islamtoleran.com',
 'label': 0,
 'PARAGRAPH': "How HugeDomains Collects this Information HugeDomains utilizes a variety of security measures to maintain the safety of customers' personal information. All supplied financial information (including credit card data) is transmitted via Secure Socket Layer (SSL) technology and then encrypted into the company's payment gateway provider's database. This database and the sensitive information contained within are only accessible by those authorized with special access rights to such systems and are required to keep the information confidential. "}